In [21]:
import pandas as pd

from datetime import datetime
import sys
sys.path.append("../")
from src.soporte_creacion_bbdd import conectar_bbdd, crear_tablas, insertar_datos
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors

# Division y preparación de las tablas

In [22]:
# Cargo el csv creado en el jupyter anterior
df_historico = pd.read_csv("../datos/raw/precios_historicos_2024-10-26.csv", index_col=0)

# Renombro la columna día por fecha porque me parece más intuitivo a la hora de realizar las siguientes busquedas
df_historico.rename(columns={"Día": "fecha"}, inplace=True)

In [23]:
# Creación del df de supermercados
"""
Aquí lo que hago es elegir la columna supermercado del df_historico, después elimino duplicados para quedarme solo con los supermercados que existen
y utilizo reset_index para darle un id temporal a cada supermercado. En este caso su id temporal va a ser su posición en la que se encuentra el primer registro.
Posteriormente lo convertiremos en un id en orden ascendente desde el 1.
Estas acciones se repiten para el resto de data frames que creamos en los pasos siguientes.
"""

df_supermercados = df_historico["Supermercado"]
df_supermercados = df_supermercados.drop_duplicates().reset_index()
df_supermercados.rename(columns={"index": "id_supermercado", "Supermercado": "supermercado"}, inplace=True)
df_supermercados

,id_supermercado,supermercado
0,0,Mercadona
1,4394,Carrefour
2,22669,Eroski
3,47459,Dia
4,60241,Hipercor
5,101423,Alcampo


In [24]:
# Creación del df de categorias
df_categorias = df_historico["Categoría"]
df_categorias = df_categorias.drop_duplicates().reset_index()
df_categorias.rename(columns={"index": "id_categoria", "Categoría": "categoria"}, inplace=True)
df_categorias

,id_categoria,categoria
0,0,Aceite de girasol
1,214,Aceite de oliva
2,1605,Leche


In [25]:
# Creación del df de productos
df_productos = df_historico["Producto"]
df_productos = df_productos.drop_duplicates().reset_index()
df_productos.rename(columns={"index": "id_producto", "Producto": "producto"}, inplace=True)
df_productos

,id_producto,producto
0,0,"Aceite De Girasol Refinado 0,2º Hacendado 1 L"
1,107,"Aceite De Girasol Refinado 0,2º Hacendado 5 L"
2,214,"Aceite De Oliva 0,4º Hacendado 1 L"
3,341,Aceite De Oliva 1º Hacendado 1 L
4,468,Aceite De Oliva Intenso Hacendado 3 L
...,...,...
1586,133766,Tierra De Sabor Leche De Vaca Entera 6 X 1 L
1587,133808,Tierra De Sabor Leche Desnatada De Vaca 1 L
1588,133851,Tierra De Sabor Leche Entera De Vaca 1 L
1589,133893,Tierra De Sabor Leche Semidesnatada De Vaca 1 L


In [26]:
# Creación del df de los registros de precios historicos
df_precios_historicos = df_historico.drop_duplicates().reset_index()
df_precios_historicos.rename(columns={"index": "id_registro", "Precio (€)": "precio", "Variación": "variacion"}, inplace=True)
df_precios_historicos

,id_registro,fecha,precio,variacion,Producto,Categoría,Supermercado
0,0,12/07/2024,1.45,=,"Aceite De Girasol Refinado 0,2º Hacendado 1 L",Aceite de girasol,Mercadona
1,1,13/07/2024,1.45,=,"Aceite De Girasol Refinado 0,2º Hacendado 1 L",Aceite de girasol,Mercadona
2,2,14/07/2024,1.45,=,"Aceite De Girasol Refinado 0,2º Hacendado 1 L",Aceite de girasol,Mercadona
3,3,15/07/2024,1.45,=,"Aceite De Girasol Refinado 0,2º Hacendado 1 L",Aceite de girasol,Mercadona
4,4,16/07/2024,1.45,=,"Aceite De Girasol Refinado 0,2º Hacendado 1 L",Aceite de girasol,Mercadona
...,...,...,...,...,...,...,...
133974,133974,22/10/2024,4.73,=,Tierra De Sabor Leche Semidesnatada De Vaca 6 ...,Leche,Alcampo
133975,133975,23/10/2024,4.73,=,Tierra De Sabor Leche Semidesnatada De Vaca 6 ...,Leche,Alcampo
133976,133976,24/10/2024,4.73,=,Tierra De Sabor Leche Semidesnatada De Vaca 6 ...,Leche,Alcampo
133977,133977,25/10/2024,4.73,=,Tierra De Sabor Leche Semidesnatada De Vaca 6 ...,Leche,Alcampo


In [27]:
# Cambio los id para que se enumeren desde el 1 en orden ascendente para todos los data frames
df_productos["id_producto"] = range(1, len(df_productos) + 1)
df_categorias["id_categoria"] = range(1, len(df_categorias) + 1)
df_supermercados["id_supermercado"] = range(1, len(df_supermercados) + 1)
df_precios_historicos["id_registro"] = range(1, len(df_precios_historicos) + 1)

In [28]:
"""
Una vez generados los nuevos ids los metemos en df_precios_historicos para posteriormente meter los datos correctamente
en la base de datos. Esto lo hacemos porque van a ser las foreign key. Además, hacemos un drop para eliminar la columna que
contiene los nombres porque no nos hacen falta. Solo queremos quedarnos con los ids.
"""

df_precios_historicos = df_precios_historicos.merge(
    df_productos[["id_producto", "producto"]],
    left_on="Producto",
    right_on="producto",
    how="left"
).drop(columns="producto")

df_precios_historicos = df_precios_historicos.merge(
    df_categorias[["id_categoria", "categoria"]],
    left_on="Categoría",
    right_on="categoria",
    how="left"
).drop(columns="categoria")

df_precios_historicos = df_precios_historicos.merge(
    df_supermercados[["id_supermercado", "supermercado"]],
    left_on="Supermercado",
    right_on="supermercado",
    how="left"
).drop(columns="supermercado")


In [29]:
# Escojo las columnas que voy a necesitar para la bbdd
df_precios_historicos_final = df_precios_historicos[["id_registro", "fecha", "precio", "variacion", "id_producto", "id_categoria", "id_supermercado"]]
df_precios_historicos_final

,id_registro,fecha,precio,variacion,id_producto,id_categoria,id_supermercado
0,1,12/07/2024,1.45,=,1,1,1
1,2,13/07/2024,1.45,=,1,1,1
2,3,14/07/2024,1.45,=,1,1,1
3,4,15/07/2024,1.45,=,1,1,1
4,5,16/07/2024,1.45,=,1,1,1
...,...,...,...,...,...,...,...
133974,133975,22/10/2024,4.73,=,1591,3,6
133975,133976,23/10/2024,4.73,=,1591,3,6
133976,133977,24/10/2024,4.73,=,1591,3,6
133977,133978,25/10/2024,4.73,=,1591,3,6


# Creación de la bbdd e inserción de los datos

In [30]:
try:
    # Nos conectamos a la base de datos
    connection = conectar_bbdd("proyecto4")
    
    # creo el cursor
    cursor = connection.cursor()

    # Creo las tablas
    crear_tablas(cursor)

    # Guardamos las tablas en la bbdd con commit
    connection.commit()

    # Si todo sale bien se ejecuta este mensaje diciendo que se han creado las tablas
    print("Se han creado las tablas")

    # Inserto los datos en las tablas por dataframe
    insertar_datos(cursor, df_supermercados, 'supermercados')
    insertar_datos(cursor, df_categorias, 'categorias')
    insertar_datos(cursor, df_productos, 'productos')
    insertar_datos(cursor, df_precios_historicos_final, 'precios_historicos')

    # Guardo los datos en la bbdd
    connection.commit()

    # Si todo sale bien se ejecuta este mensaje de confirmación
    print("Se han guardado los datos")
    
except Exception as error:
    print(f"Ocurrió un error: {error}")
    connection.rollback()
finally:
    # Finalmente cerramos el cursor y la conexión a la bbdd
    if cursor:
        cursor.close()
    if connection:
        connection.close()
    print("Conexión cerrada")

Se han creado las tablas
Se han guardado los datos
Conexión cerrada
